<a href="https://colab.research.google.com/github/Anil951/Early-detection-of-mental-health/blob/main/implementation_step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pytesseract

In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,611 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,456 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,4

In [ ]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [ ]:
!pip install pytesseract

In [ ]:
import pandas as pd
import re
from PIL import Image
import pytesseract

image = Image.open('/content/report_card_class10.jpg')

text = pytesseract.image_to_string(image).lower()

def clean(subject):
    return re.sub(r"[^0-9a-zA-Z\s]", "", subject).strip()

lines = [line.strip() for line in text.split('\n') if ':' not in line and 'total' not in line]
for i in lines:
  if(len(i)==0):
      lines.remove(i)

lines = [clean(line) for line in lines if clean(line)]



print(lines)

['term1', 'subject', 'math', 'sclence', 'english', 'history', 'physical education', 'score', '90', '85', '88', '80', '95', 'attendance', '92', '88', '90', '93', '97', 'remarks', 'consistently performs well shows strong skils', 'active in discussions good understanding of concepts', 'strong kills participates actively in group work', 'engaged in class discussions needs improvement', 'high energy and excellent team player', 'term2', 'subject', 'math', 'sclence', 'english', 'history', 'physical education', 'score', '85', '78', '80', '72', '90', 'attendance', '89', '84', '85', '88', '92', 'remarks', 'shows occasional struggles  needs extra practice', 'lacks motivation in class needs more focus', 'reduced participation distracted in class', 'difficulties in remembering needs to engage more', 'should improve fitness level', 'term3', 'subject', 'math', 'sclence', 'english', 'history', 'physical education', 'score', '75', '70', '72', '65', '80', 'attendance', '83', '75', '78', '80', '85', 'rem

In [ ]:
terms = {}
data=lines
i = 0
while i < len(data):
    if 'term' in data[i]:
        term_name = data[i]
        terms[term_name] = {}
        i += 1
        
        while i < len(data) and 'term' not in data[i]:
            col_name = data[i]
            col_values = data[i + 1:i + 6]
            terms[term_name][col_name] = col_values
            i += 6  
        
    else:
        i += 1 

dfs = {term: pd.DataFrame(data) for term, data in terms.items()}
dfs

{'term1':               subject score attendance  \
 0                math    90         92   
 1             sclence    85         88   
 2             english    88         90   
 3             history    80         93   
 4  physical education    95         97   
 
                                              remarks  
 0      consistently performs well shows strong skils  
 1  active in discussions good understanding of co...  
 2   strong kills participates actively in group work  
 3     engaged in class discussions needs improvement  
 4              high energy and excellent team player  ,
 'term2':               subject score attendance  \
 0                math    85         89   
 1             sclence    78         84   
 2             english    80         85   
 3             history    72         88   
 4  physical education    90         92   
 
                                             remarks  
 0  shows occasional struggles  needs extra practice  
 1        lacks

In [ ]:
dfs['term3']

,subject,score,attendance,remarks
0,math,75,83,struggles with advanced topics seems frustrated
1,sclence,70,75,no interest in class lab work is neglected
2,english,72,78,very little participation often distracted
3,history,65,80,limited engagement affecting performance
4,physical education,80,85,results decreased tired uninterested


In [ ]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

nltk.download("vader_lexicon")
sid = SentimentIntensityAnalyzer()

term_data = dfs


avg_term1 = term_data['term1'][['score', 'attendance', 'senti']].mean()
avg_term3 = term_data['term3'][['score', 'attendance', 'senti']].mean()

score_decrease = ((avg_term1['score'] - avg_term3['score']) / avg_term1['score']) * 100
attendance_decrease = ((avg_term1['attendance'] - avg_term3['attendance']) / avg_term1['attendance']) * 100
senti_decrease = avg_term1['senti'] - avg_term3['senti']

overall_decrease = pd.DataFrame({
    'Metric': ['Score', 'Attendance', 'Sentiment'],
    'Decrease (%)': [score_decrease, attendance_decrease, senti_decrease * 100]
})

print(overall_decrease)


       Metric  Decrease (%)
0       Score     17.351598
1  Attendance     12.826087
2   Sentiment     88.694000
There is a high likelihood of mental health issues based on the observed decreases.


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
